# Tracking data

Metadata is available on tracking data. TRACAB and Metrica does not include any extra metadata besides team and jersey number. [EPTS](https://football-technology.fifa.com/en/media-tiles/research-development-epts-standard-data-format/) is the only format right now that contains additional information that will be loaded as metadata


## Loading Metrica data


In [2]:
from kloppy import metrica

dataset = metrica.load_open_data(limit=10)


## Exploring the data

When you want to show the name of a player you are advised to use `str(player)`. This will call the magic `__str__` method that handles fallbacks for missing data. By default it will return `full_name`, and fallback to 1) `first_name last_name` 2) `player_id`.

In [2]:
metadata = dataset.metadata
home_team, away_team = metadata.teams

[f"{player} ({player.jersey_no})" for player in home_team.players]


['home_11 (11)',
 'home_1 (1)',
 'home_2 (2)',
 'home_3 (3)',
 'home_4 (4)',
 'home_5 (5)',
 'home_6 (6)',
 'home_7 (7)',
 'home_8 (8)',
 'home_9 (9)',
 'home_10 (10)',
 'home_12 (12)',
 'home_13 (13)',
 'home_14 (14)']

In [3]:
print(f"{home_team.ground} - {home_team}")
print(f"{away_team.ground} - {away_team}")

home - Home
away - Away


## Working with tracking data

The actual tracking data is available at `dataset.frames`. This list holds all frames. Each frame has a `players_coordinates` dictionary that is indexed by `Player` entities and has values of the `Point` type.



In [4]:
first_frame = dataset.frames[0]
print(f"Number of players in the frame: {len(first_frame.players_coordinates)}")

from pprint import pprint
print("List home team players coordinates")
pprint([
    player_coordinates 
    for player, player_coordinates
    in first_frame.players_coordinates.items()
    if player.team == home_team
])

Number of players in the frame: 22
List home team players coordinates
[Point(x=0.00082, y=0.51762),
 Point(x=0.32648, y=0.34678),
 Point(x=0.33701, y=0.51137),
 Point(x=0.30927, y=0.64471),
 Point(x=0.32137, y=0.78738),
 Point(x=0.41094, y=0.27410999999999996),
 Point(x=0.41698, y=0.52157),
 Point(x=0.39125, y=0.6745),
 Point(x=0.45388, y=0.78826),
 Point(x=0.52697, y=0.6202),
 Point(x=0.55243, y=0.56731)]


## Exporting to a dataframe

kloppy allows you to export a `dataset` to a `DataFrame`. Both polars and pandas are supported. You can use the following engines: `polars`, `pandas`, `pandas[pyarrow]`. The pyarrow engine requires python 3.8 or higher.

In [13]:
df = dataset.to_df(engine="pandas")  # default behaviour
df.dtypes

period_id                int64
timestamp              float64
frame_id                 int64
ball_state              object
ball_owning_team_id     object
                        ...   
away_23_s               object
away_24_x              float64
away_24_y              float64
away_24_d               object
away_24_s               object
Length: 96, dtype: object

In [14]:
df = dataset.to_df(engine="pandas[pyarrow]")
df.dtypes

period_id               int64[pyarrow]
timestamp              double[pyarrow]
frame_id                int64[pyarrow]
ball_state               null[pyarrow]
ball_owning_team_id      null[pyarrow]
                            ...       
away_23_s                null[pyarrow]
away_24_x              double[pyarrow]
away_24_y              double[pyarrow]
away_24_d                null[pyarrow]
away_24_s                null[pyarrow]
Length: 96, dtype: object

In [15]:
df

,period_id,timestamp,frame_id,ball_state,ball_owning_team_id,ball_x,ball_y,ball_z,home_11_x,home_11_y,...,away_22_d,away_22_s,away_23_x,away_23_y,away_23_d,away_23_s,away_24_x,away_24_y,away_24_d,away_24_s
0,1,0.00,1,<NA>,<NA>,0.45472,0.61291,<NA>,0.00082,0.51762,...,<NA>,<NA>,0.43693,0.94998,<NA>,<NA>,0.37833,0.72617,<NA>,<NA>
1,1,0.04,2,<NA>,<NA>,0.49645,0.59344,<NA>,0.00096,0.51762,...,<NA>,<NA>,0.43693,0.94998,<NA>,<NA>,0.37833,0.72617,<NA>,<NA>
2,1,0.08,3,<NA>,<NA>,0.53716,0.57444,<NA>,0.00114,0.51762,...,<NA>,<NA>,0.43693,0.94998,<NA>,<NA>,0.37833,0.72617,<NA>,<NA>
3,1,0.12,4,<NA>,<NA>,0.55346,0.57769,<NA>,0.00121,0.51762,...,<NA>,<NA>,0.43644,0.94962,<NA>,<NA>,0.37756,0.72527,<NA>,<NA>
4,1,0.16,5,<NA>,<NA>,0.55512,0.59430,<NA>,0.00129,0.51762,...,<NA>,<NA>,0.43580,0.95023,<NA>,<NA>,0.37663,0.72457,<NA>,<NA>
5,1,0.20,6,<NA>,<NA>,0.55677,0.61091,<NA>,0.00140,0.51762,...,<NA>,<NA>,0.43504,0.95116,<NA>,<NA>,0.37557,0.72327,<NA>,<NA>
6,1,0.24,7,<NA>,<NA>,0.55842,0.62752,<NA>,0.00140,0.51762,...,<NA>,<NA>,0.43423,0.95195,<NA>,<NA>,0.37439,0.72249,<NA>,<NA>
7,1,0.28,8,<NA>,<NA>,0.56004,0.64371,<NA>,0.00140,0.51762,...,<NA>,<NA>,0.43347,0.95343,<NA>,<NA>,0.37320,0.72152,<NA>,<NA>
8,1,0.32,9,<NA>,<NA>,0.56173,0.66074,<NA>,0.00120,0.51807,...,<NA>,<NA>,0.43273,0.95422,<NA>,<NA>,0.37196,0.72073,<NA>,<NA>
9,1,0.36,10,<NA>,<NA>,0.56338,0.67735,<NA>,0.00100,0.51853,...,<NA>,<NA>,0.43199,0.95566,<NA>,<NA>,0.37068,0.71990,<NA>,<NA>


In [16]:
df = dataset.to_df(engine="polars")
df

period_id,timestamp,frame_id,ball_state,ball_owning_team_id,ball_x,ball_y,ball_z,home_11_x,home_11_y,home_11_d,home_11_s,home_1_x,home_1_y,home_1_d,home_1_s,home_2_x,home_2_y,home_2_d,home_2_s,home_3_x,home_3_y,home_3_d,home_3_s,home_4_x,home_4_y,home_4_d,home_4_s,home_5_x,home_5_y,home_5_d,home_5_s,home_6_x,home_6_y,home_6_d,home_6_s,home_7_x,...,away_15_s,away_16_x,away_16_y,away_16_d,away_16_s,away_17_x,away_17_y,away_17_d,away_17_s,away_18_x,away_18_y,away_18_d,away_18_s,away_19_x,away_19_y,away_19_d,away_19_s,away_20_x,away_20_y,away_20_d,away_20_s,away_21_x,away_21_y,away_21_d,away_21_s,away_22_x,away_22_y,away_22_d,away_22_s,away_23_x,away_23_y,away_23_d,away_23_s,away_24_x,away_24_y,away_24_d,away_24_s
i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,...,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,0.0,1,null,null,0.45472,0.61291,null,0.00082,0.51762,null,null,0.32648,0.34678,null,null,0.33701,0.51137,null,null,0.30927,0.64471,null,null,0.32137,0.78738,null,null,0.41094,0.27411,null,null,0.41698,0.52157,null,null,0.39125,...,null,0.67658,0.5329,null,null,0.6731,0.23524,null,null,0.40783,0.38475,null,null,0.45472,0.61291,null,null,0.5596,0.32225,null,null,0.55243,0.56731,null,null,0.50067,0.05678,null,null,0.43693,0.94998,null,null,0.37833,0.72617,null,null
1,0.04,2,null,null,0.49645,0.59344,null,0.00096,0.51762,null,null,0.32648,0.34678,null,null,0.33701,0.51137,null,null,0.30927,0.64471,null,null,0.32137,0.78738,null,null,0.41094,0.27411,null,null,0.41698,0.52157,null,null,0.39125,...,null,0.67658,0.5329,null,null,0.6731,0.23524,null,null,0.40783,0.38475,null,null,0.45472,0.61291,null,null,0.5596,0.32225,null,null,0.55243,0.56731,null,null,0.50067,0.05678,null,null,0.43693,0.94998,null,null,0.37833,0.72617,null,null
1,0.08,3,null,null,0.53716,0.57444,null,0.00114,0.51762,null,null,0.32648,0.34678,null,null,0.33701,0.51137,null,null,0.30927,0.64471,null,null,0.32137,0.78738,null,null,0.41094,0.27411,null,null,0.41698,0.52157,null,null,0.39125,...,null,0.67658,0.5329,null,null,0.6731,0.23524,null,null,0.40783,0.38475,null,null,0.45472,0.61291,null,null,0.5596,0.32225,null,null,0.55243,0.56731,null,null,0.50067,0.05678,null,null,0.43693,0.94998,null,null,0.37833,0.72617,null,null
1,0.12,4,null,null,0.55346,0.57769,null,0.00121,0.51762,null,null,0.32622,0.34683,null,null,0.33687,0.51012,null,null,0.30944,0.64446,null,null,0.32142,0.78759,null,null,0.41067,0.27435,null,null,0.41703,0.52035,null,null,0.39134,...,null,0.6764,0.53238,null,null,0.67279,0.23458,null,null,0.40771,0.38495,null,null,0.45454,0.61182,null,null,0.55974,0.32224,null,null,0.55236,0.56687,null,null,0.50034,0.05609,null,null,0.43644,0.94962,null,null,0.37756,0.72527,null,null
1,0.16,5,null,null,0.55512,0.5943,null,0.00129,0.51762,null,null,0.32597,0.34731,null,null,0.33664,0.50982,null,null,0.30948,0.64472,null,null,0.32136,0.78841,null,null,0.41065,0.2749,null,null,0.41682,0.52048,null,null,0.39141,...,null,0.67599,0.53231,null,null,0.67253,0.23436,null,null,0.40736,0.38529,null,null,0.45426,0.61275,null,null,0.55992,0.32209,null,null,0.55202,0.56689,null,null,0.50017,0.05566,null,null,0.4358,0.95023,null,null,0.37663,0.72457,null,null
1,0.2,6,null,null,0.55677,0.61091,null,0.0014,0.51762,null,null,0.32581,0.34841,null,null,0.33637,0.51034,null,null,0.30978,0.64521,null,null,0.32159,0.78949,null,null,0.41082,0.27561,null,null,0.41663,0.5218,null,null,0.3919,...,null,0.67537,0.53273,null,null,0.67194,0.23481,null,null,0.40697,0.38691,null,null,0.45397,0.61379,null,null,0.56017,0.32324,null,null,0.55157,0.56743,null,null,0.49989,0.05568,null,null,0.43504,0.95116,null,null,0.37557,0.72327,null,null
1,0.24,7,null,null,0.55842,0.62752,null,0.0014,0.51762,null,null,0.3257,0.34963,null,null,0.33624,0.51081,null,null,0.31015,0.64561,null,null,0.32198,0.79079,nu